In [46]:
import pandas as pd


In [49]:
df = pd.read_excel("trafikKaza.xlsx")

In [50]:
df.head()

,KAZA TURU,KAZA TARIHI,AY,MEVSİM,KAZA SAAT,ILCE,MAHALLE,YOL,OLU SAYISI,YARALI SAYISI,...,OLUS NEDENI,KAPLAMA CINSI,SERIT SAYISI,AYDINLATMA,YASAL HIZ LIMITI,ARAC CINSI,ARAC MODELI,SURUCU CINSIYET,SURUCU EGITIM DURUMU,EHLIYET YILI
0,Yaralamali,2013-01-02,1,kis,0.0,MELIKGAZI,KOSK,TALAS BULV.,NaN,1.0,...,Duran araca carpma,Asfalt,3,Var,50,Otomobil,1992,KADIN,ILKOKUL,1
1,Yaralamali,2013-01-02,1,kis,17.0,MELIKGAZI,OSMAN KAVUNCU,OSMAN KAVUNCU BULV.,NaN,1.0,...,Arkadan carpma,Asfalt,3,Var,50,Otomobil,2001,ERKEK,2 YIL SURELI YUKSEK,2
2,Yaralamali,2013-01-03,1,kis,20.0,MELIKGAZI,YILDIRIM BEYAZIT,ARIF CD.,NaN,1.0,...,Yandan carpma,Asfalt,3,Var,50,Minibus,1987,ERKEK,ILKOKUL,4
3,Yaralamali,2013-01-04,1,kis,7.0,TALAS,HARMAN,NaN,NaN,10.0,...,Yandan carpma,Asfalt,2,Yok,50,Minibus,2001,ERKEK,ILKOGRETIM,4
4,Yaralamali,2013-01-04,1,kis,16.0,MELIKGAZI,EGRIBUCAK,EGRIBUCAK CD.,NaN,6.0,...,Arkadan carpma,Asfalt,2,Yok,70,Otomobil,1998,ERKEK,2 YIL SURELI YUKSEK,3


In [51]:
df.isnull().sum()

KAZA TURU                   0
KAZA TARIHI                 0
AY                          0
MEVSİM                      0
KAZA SAAT                  27
ILCE                        1
MAHALLE                    58
YOL                      1256
OLU SAYISI              15109
YARALI SAYISI              37
TASIT SAYISI               12
SURUCU KUSURU               0
OLUS NEDENI                 8
KAPLAMA CINSI               0
SERIT SAYISI                0
AYDINLATMA                  0
YASAL HIZ LIMITI            0
ARAC CINSI                  0
ARAC MODELI                 0
SURUCU CINSIYET             0
SURUCU EGITIM DURUMU        0
EHLIYET YILI                0
dtype: int64

In [52]:
df = df.dropna(axis=0)

In [53]:
df.isnull().sum()

KAZA TURU               0
KAZA TARIHI             0
AY                      0
MEVSİM                  0
KAZA SAAT               0
ILCE                    0
MAHALLE                 0
YOL                     0
OLU SAYISI              0
YARALI SAYISI           0
TASIT SAYISI            0
SURUCU KUSURU           0
OLUS NEDENI             0
KAPLAMA CINSI           0
SERIT SAYISI            0
AYDINLATMA              0
YASAL HIZ LIMITI        0
ARAC CINSI              0
ARAC MODELI             0
SURUCU CINSIYET         0
SURUCU EGITIM DURUMU    0
EHLIYET YILI            0
dtype: int64

In [61]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

scatter_matrix(df, figsize=(15, 12), alpha=1)
plt.tight_layout()
plt.show()
//


SyntaxError: invalid syntax (3258438563.py, line 7)